In [1]:
import pandas as pd, numpy as np
import json

In [2]:
progress = pd.read_csv('historical_progresses_aug.csv')

In [3]:
progress[progress['subjectCode'] == 'en'].groupby(['subjectCode', 'studentName', 'lessonId']).agg({'score': [np.max], 'lessonOrder': lambda x: np.mean(x)})

score lessonOrder
                                  amax    <lambda>
subjectCode studentName lessonId                  
en          Aadharsha   en0500      83        59.0
            Abhisheck S en0203      48        23.0
                        en0204     100        24.0
            Afishabhnu  balloon6   100        62.0
                        en0500     100        59.0
...                                ...         ...
            kavitha     en2400      25       236.0
                        en2600      42       258.0
                        en2800      36       267.0
                        en2801      42       268.0
                        en3300      62       295.0

[2281 rows x 2 columns]

In [4]:
students = {}
for row in progress.itertuples():
    score = row[7]
    student = row[10]
    subject = row[12]
    lesson_order = row[14]
    date_time = row[13]
    if(student == 'nan' or student == '2'):
        print(row)
    if student not in students:
        students[student] = {subject: {lesson_order: (score, date_time)}}
    elif subject not in students[student]:
        students[student][subject] = {lesson_order: (score, date_time)}
    elif lesson_order not in students[student][subject]:
        students[student][subject][lesson_order] = (score, date_time)
    elif students[student][subject][lesson_order][0] < score:
        students[student][subject][lesson_order] = (score, date_time)


Pandas(Index=12353, id='apKOOJTvvKXj4v5e5x8C', progressId='dade2ce8-7c3c-409c-9583-c50b2ee17a8a', chapterId='maths02', chapterName='Numbers 6 -10', lessonId='maths0200', lessonName='6', score=100, schoolId='fnjovAUCZBgpZRh3Iwcv', sectionId='6SHA2IdrZL3l1u3incGL', studentId='2', studentName='Chandan ', subjectCode='maths', date='Thu Aug 25 2022 13:53:56 GMT+0530 (India Standard Time)', lessonOrder=12)
Pandas(Index=12354, id='5OOH8op7WGhRpE1fqA1C', progressId='dade2ce8-7c3c-409c-9583-c50b2ee17a8a', chapterId='maths02', chapterName='Numbers 6 -10', lessonId='maths0201', lessonName='7', score=100, schoolId='fnjovAUCZBgpZRh3Iwcv', sectionId='6SHA2IdrZL3l1u3incGL', studentId='2', studentName='Chandan ', subjectCode='maths', date='Thu Aug 25 2022 13:53:57 GMT+0530 (India Standard Time)', lessonOrder=13)
Pandas(Index=12355, id='KVCGyP4RRlgRrlDlcvyl', progressId='dade2ce8-7c3c-409c-9583-c50b2ee17a8a', chapterId='maths02', chapterName='Numbers 6 -10', lessonId='maths0202', lessonName='8', score=

In [5]:
print(students.keys())

dict_keys(['CV01', 'CV08', 'CV09', 'CV02', 'CV04', 'CV03', nan, 'CV10', 'CV07', 'CV06', 'KU18', 'KU14', 'KU16', 'KU21', 'KU15', 'KU11', 'KU17', 'KU20', 'KU19', 'KU09', 'KU12', 'KD16', 'KD17', 'KU07', 'KU02', 'KU08', 'KD15', 'KU01', 'KU06', 'KU05', 'SH31', 'SH29', 'SH20', 'SH22', 'SH28', 'SH19', 'SH30', 'SH18', 'SH32', 'SH27', 'SH24', 'SH17', 'SH09', 'SH10', 'SH14', 'SH08', 'SH04', 'SH07', 'SH13', 'SH16', 'SH12', 'SH01', 'SH05', 'SH03', 'SH02', 'KH24', 'KH20', 'KH25', 'KH18', 'KH23', 'KH27', 'KH30', 'KH33', 'KH34', 'KH29', 'KH28', 'KH35', 'KH21', 'KH19', 'KH03', 'KH09', 'KH13', 'KH04', 'KH08', 'KH01', 'KH06', 'KH05', 'KH02', 'KH10', 'KH17', 'KH14', 'KH15', '1', 'MA24', 'MA06', 'MA27', 'MA26', 'MA25', 'MA03', 'MA08', 'MA16', 'aq2', 'MA13', 'MA15', 'MA21', 'MA17', 'MA22', 'awsd', 'MA10', 'MA14', 'MA18', 'MA12', 'BA35', 'BA22', 'BA31', 'BA21', 'BA20', 'BA24', 'BA23', 'BA28', 'BA36', 'BA34', 'BA26', 'BA03', 'BA02', 'BA12', 'BA13', 'BA06', 'BA17', 'BA04', 'BA07', 'BA11', 'BA15', 'BA05', 'BA1

In [8]:
max_lessons = {
    'kn': 379/2,
    'en': 303/2,
    'maths': 404/2
}
# 1, 1.5, 2.5, 3, 4, 5, 6,7,7.1,7.2,7.3,7.5,7.7
month = 7.7
total_months = 10
alpha_avg = 0
for student, course in students.items():
    course_avg = {
        'kn': 0,
        'en': 0,
        'maths': 0
    }
    lesson_details = []
    for course_name, lessons in course.items():
        if course_name in max_lessons:
            lesson_avg = 0
            score_avg = 0
            for lesson_order, score in lessons.items():
                lesson_avg += lesson_order
                score_avg += score[0]
            lesson_avg = lesson_avg / max(max_lessons[course_name]/total_months, len(lessons))
            ideal_lesson_avg = max_lessons[course_name] * (month - 0.5)/total_months
            score_avg /= len(lessons)
            # print(course_name, lesson_avg, ideal_lesson_avg, lesson_avg/ideal_lesson_avg, score_avg, len(lessons), max_lessons[course_name]/total_months)
            course_avg[course_name] = lesson_avg/ideal_lesson_avg * (score_avg + 15) / 100
            lesson_details.append({'course': course_name, 'lesson_num': lesson_order, 'score': score[0], 'date': score[1]})
    alpha = (course_avg['kn'] + course_avg['en'] + course_avg['maths']) / 3
    print(student, '\t', alpha, '\t', json.dumps(lesson_details))
    alpha_avg += alpha
alpha_avg = alpha_avg / len(students)

CV01 	 0.12660380757984127 	 [{"course": "en", "lesson_num": 67, "score": 90, "date": "Thu Aug 04 2022 11:43:32 GMT+0530 (India Standard Time)"}, {"course": "kn", "lesson_num": 31, "score": 30, "date": "Wed Aug 10 2022 10:28:16 GMT+0530 (India Standard Time)"}, {"course": "maths", "lesson_num": 252, "score": 4, "date": "Sat Aug 20 2022 10:07:30 GMT+0530 (India Standard Time)"}]
CV08 	 0.15771775083384448 	 [{"course": "en", "lesson_num": 46, "score": 80, "date": "Wed Aug 17 2022 14:51:14 GMT+0530 (India Standard Time)"}, {"course": "kn", "lesson_num": 16, "score": 52, "date": "Thu Aug 18 2022 14:25:27 GMT+0530 (India Standard Time)"}, {"course": "maths", "lesson_num": 60, "score": 100, "date": "Fri Aug 26 2022 11:14:35 GMT+0530 (India Standard Time)"}]
CV09 	 0.2661550270126023 	 [{"course": "en", "lesson_num": 48, "score": 95, "date": "Thu Aug 04 2022 12:52:22 GMT+0530 (India Standard Time)"}, {"course": "maths", "lesson_num": 27, "score": 98, "date": "Fri Aug 26 2022 10:56:04 GMT+053

In [9]:
print(alpha_avg)

0.32059744326665124
